In [1]:
#pip install yfinance --upgrade --no-cache-dir
#pip install psycopg2
#pip install SQLAlchemy

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/14/03/3c854ca3e02eedf614abba4b2e177c469bf3af58207fa30d5098c5d652fe/yfinance-0.2.37-py2.py3-none-any.whl.metadata
  Obtaining dependency information for multitasking>=0.0.7 from https://files.pythonhosted.org/packages/3e/8a/bb3160e76e844db9e69a413f055818969c8acade64e1a9ac5ce9dfdcf6c1/multitasking-0.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for pytz>=2022.5 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 964.8 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 342.4 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requi

In [12]:
import yfinance as yf
import pandas as pd
import psycopg2
from getpass import getpass
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [13]:
tickers = yf.Tickers('msft aapl goog tsla amzn meta vz cost wmt nke')
#agregar 4 más

In [50]:

companies = ('MSFT','AAPL','GOOG','TSLA','AMZN','META','VZ','COST','WMT','NKE')
historical = pd.DataFrame()
for i in  range(len(companies)):
    df = tickers.tickers[companies[i]].history(period="5y")
    df['symbol'] = tickers.tickers[companies[i]].info['symbol']
    df['short_name'] = tickers.tickers[companies[i]].info['shortName']
    df['long_name'] = tickers.tickers[companies[i]].info['longName']
    df['address'] = tickers.tickers[companies[i]].info['address1']
    df['city'] = tickers.tickers[companies[i]].info['city']
    df['state'] = tickers.tickers[companies[i]].info['state']
    df['zip'] = tickers.tickers[companies[i]].info['zip']
    df['country'] = tickers.tickers[companies[i]].info['country']
    df['phone'] = tickers.tickers[companies[i]].info['phone']
    df['website'] = tickers.tickers[companies[i]].info['website']
    df['industry'] = tickers.tickers[companies[i]].info['industry']
    df['sector'] = tickers.tickers[companies[i]].info['sector']
    historical = pd.concat([historical,df], ignore_index=False)
historical = historical.reset_index()
subrogate_key = historical['Date'].apply(lambda x: dt.datetime.strftime(x,"%Y%m%d%H%M%S")) + historical['symbol']
historical['stock_id'] = subrogate_key
historical['upload_date'] = dt.datetime.date(dt.datetime.now())
historical['Date'] = historical['Date'].apply(lambda x: dt.datetime.strptime(dt.datetime.strftime(x,"%Y-%m-%d"),"%Y-%m-%d"))
historical['Date'] = historical['Date'].apply(lambda x: dt.datetime.date(x))
historical = historical.rename(columns={'Open':'open_value', 'Close':'close_value','High':'high_value', 'Low':'low_value', 'Stock Splits':'stock_splits'})


In [15]:
historical.head()

,Date,open_value,high_value,low_value,close_value,Volume,Dividends,stock_splits,symbol,short_name,...,city,state,zip,country,phone,website,industry,sector,upload_date,stock_id
0,2019-04-22,116.721034,118.034643,116.673436,117.806190,15648700,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,2024-04-21,20190422000000MSFT
1,2019-04-23,118.129848,119.538652,117.872840,119.405388,24025500,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,2024-04-21,20190423000000MSFT
2,2019-04-24,119.738530,119.795642,118.529623,118.996056,31257000,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,2024-04-21,20190424000000MSFT
3,2019-04-25,123.803107,125.050084,122.632284,122.936882,38033900,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,2024-04-21,20190425000000MSFT
4,2019-04-26,123.460435,124.240993,122.813155,123.641296,23654900,0.0,0.0,MSFT,Microsoft Corporation,...,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software - Infrastructure,Technology,2024-04-21,20190426000000MSFT


In [59]:
host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="gustavo_hiram_gutierrez_coderhouse"
pwd = 'T93hbU4sqc' #getpass()

try:
    conn = psycopg2.connect(
        host=host,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conexión a Redshift Existosa!")
    
except Exception as e:
    print("No es posible conectarse a Redshift")
    print(e)

Conexión a Redshift Existosa!


In [23]:
create_table = ''' CREATE TABLE IF NOT EXISTS gustavo_hiram_gutierrez_coderhouse.stock_market  (
    stock_id varchar(25) NOT NULL PRIMARY KEY,
    symbol VARCHAR(100) NOT NULL,
    short_name VARCHAR(100) NOT NULL,
    long_name VARCHAR(100) NOT NULL,
    address VARCHAR(100) NOT NULL,
    city VARCHAR(100) NOT NULL,
    state VARCHAR(100) NOT NULL,
    zip VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    phone VARCHAR(100) NOT NULL,
    website VARCHAR(100) NOT NULL,
    industry VARCHAR(100) NOT NULL,
    sector VARCHAR(100) NOT NULL,
    date date NOT NULL,
    open_value DECIMAL NOT NULL,
    high_value DECIMAL NOT NULL,
    low_value DECIMAL NOT NULL,
    close_value DECIMAL NOT NULL,
    volume integer NOT NULL,
    dividends DECIMAL NOT NULL,
    stock_splits DECIMAL NOT NULL,
    upload_date date NOT NULL
)'''

with conn.cursor() as cur:
    cur.execute(create_table)
    conn.commit()
print('Tabla Creada de Manera Exitosa!')

Tabla Creada de Manera Exitosa!


# Checar última fecha en la base de datos

In [55]:
with conn.cursor() as cur:
    cur.execute('SELECT MAX(date) FROM gustavo_hiram_gutierrez_coderhouse.stock_market')
    results = cur.fetchall()
    max_date = results[0][0]
    print('Last date:',results[0][0])
historical = historical[historical['Date']> max_date]

Last date: 2024-03-28


In [ ]:
#Existen dos Métodos de ingestar la data a Redshift

# Método 1
conn_string = URL.create(
    'postgresql',
    username=user,
    password=pwd,
    host=host,
    port='5439',
    database=data_base)
engine = create_engine(conn_string)
with engine.connect() as conn:
    companies = ('MSFT','AAPL','GOOG','TSLA','AMZN','META','VZ','COST','WMT','NKE')
    for i in  range(len(companies)):
        engine = create_engine(conn_string)
        historical[historical['symbol']== companies[i]].to_sql('gustavo_hiram_gutierrez_coderhouse.stock_market1', con=conn, index=False, if_exists='append')
        print('Data correctamente insertada para ', companies[i])
print('Proceso Terminado de manera Exitosa')

In [60]:
# Método 2
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO gustavo_hiram_gutierrez_coderhouse.stock_market (Date, open_value, high_value, low_value,
          close_value, Volume, Dividends, stock_splits,symbol,short_name,long_name,address,city,state,zip,country,phone,
          website,industry,sector,stock_id,upload_date)
        VALUES %s
        ''',
        [tuple(row) for row in historical.values],
        page_size=len(historical)
    )
    conn.commit()
print('Data correctamente insertada!')

Data correctamente insertada!
